In [368]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
# % matplotlib inline
#plt.rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML

from copy import deepcopy
import os
import sys
from functools import reduce
sys.path.append("/home/ful7rng/projects/transition/propagation")
from utils import pickle_load, get_npy, get_file
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space
from utils.scene_utils import display_scenes_trajs, animate_scenes, get_scenes
from propagation.bofum import conditionalBOFUM, naiveBOFUM, BOFUMRealdata
from propagation.animation import TrackingAnimRealdata, Plot, TrackingAnimSimulation
from data_loader import get_map_crop


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
plt.ion()

In [117]:
from utils.plot_utils import plot_4d_tensor

def onclick(event, models, anim, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
    print(coords)
    
    all_axes = [plot.axes for plot in anim.plots]
    
    for i, ax in enumerate(all_axes):

        # For infomation, print which axes the click was in
        if ax == event.inaxes:
            print "Click is in axes ax{}".format(i+1)
            break
            
    clicked = np.zeros_like(models[0].map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    for plot in anim.plots:
        plot.set_axes_data("occupancy_axes", clicked)
    anim.fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    
    
    if models[i].kernels.ndim == 6:
        kernel = models[i].kernels[x, y]
        condi_prob = models[i].nn_probs[x, y]
    else:
        kernel = models[i].kernels
        condi_prob = models[i].kernels
        
#     print(kernel.shape)
#     print(condi_prob.shape)
    print(np.sum(kernel, axis=(2, 3)))
    plot_4d_tensor(condi_prob, fig=fig_1)
    plot_4d_tensor(kernel, fig=fig_2)
    if hasattr(models[i], 'apply_vel_mask') and models[i].apply_vel_mask:
        print("plot vel mask")
        ma_plot.set_axes_data("occupancy_axes", models[i].vel_mask[x, y])
    else:
        ma_plot.set_axes_data("occupancy_axes", models[i].ma_vel)
    vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt_pred[x, y])
    merge_vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt_merged[x, y])
    final_vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt[x, y])
    #print(models[i].P_Vt[x, y])
    fig_1.canvas.draw()
    fig_2.canvas.draw()
    fig_3.canvas.draw()
    
    model_names = map(lambda model: model.name, models)
    occs = map(lambda model: model.P_Ot[x, y], models)
    for name, occ in zip(model_names, occs):
        print("loc ({}, {}) of model {} has occupancy of {}".format(x, y, name, occ))

In [320]:
from utils.plot_utils import plot_4d_tensor

def onclick_keep_motion(event, models, anim, fig_1, fig_2, fig_3, ma_plot, vel_plot, bofmp_vel_plot, keep_motion_plot):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
    print(coords)
    
    all_axes = [plot.axes for plot in anim.plots]
    
    for i, ax in enumerate(all_axes):

        # For infomation, print which axes the click was in
        if ax == event.inaxes:
            print "Click is in axes ax{}".format(i+1)
            break
            
    clicked = np.zeros_like(models[0].map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    for plot in anim.plots:
        plot.set_axes_data("occupancy_axes", clicked)
    anim.fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    
    
    if models[i].kernels.ndim == 6:
        kernel = models[i].kernels[x, y]
        condi_prob = models[i].nn_probs[x, y]
    else:
        kernel = models[i].kernels
        condi_prob = models[i].kernels
        
#     print(kernel.shape)
#     print(condi_prob.shape)
    print(np.sum(kernel, axis=(2, 3)))
    plot_4d_tensor(condi_prob, fig=fig_1)
    plot_4d_tensor(kernel, fig=fig_2)
    if hasattr(models[i], 'apply_vel_mask') and models[i].apply_vel_mask:
        print("plot vel mask")
        ma_plot.set_axes_data("occupancy_axes", models[i].vel_mask[x, y])
    else:
        ma_plot.set_axes_data("occupancy_axes", models[i].ma_vel)
    vel_plot.set_axes_data("occupancy_axes", models[i].P_Vt_pred[x, y])
    merge_vel_plot.set_axes_data("occupancy_axes", models[1].P_Vt[x, y])
    final_vel_plot.set_axes_data("occupancy_axes", models[2].P_Vt[x, y])
    #print(models[i].P_Vt[x, y])
    fig_1.canvas.draw()
    fig_2.canvas.draw()
    fig_3.canvas.draw()
    
    model_names = map(lambda model: model.name, models)
    occs = map(lambda model: model.P_Ot[x, y], models)
    for name, occ in zip(model_names, occs):
        print("loc ({}, {}) of model {} has occupancy of {}".format(x, y, name, occ))

In [150]:
def on_press(event, animation):
    global scenes
    if event.key == 'n':
        idx = np.random.choice(np.arange(len(scenes)))
        print(idx)
        scene = scenes[idx]
        animation.update(scene)

In [119]:
diagonal = True
conditional = True
if diagonal:
    if conditional:
        model_name = '04_DIAGONAL_TRUE_CONDITIONAL_TRUE'
    else:
        model_name = '04_DIAGONAL_TRUE_CONDITIONAL_FALSE'
else:
    # model_name = '20_ALL_MAPS_W_MASK'
    if conditional:
        model_name = '03_DIAGONAL_FALSE_CONDITIONAL_TRUE'
    else:
        model_name = '04_DIAGONAL_FALSE_CONDITIONAL_FALSE'

In [344]:
fname = '/local/data/scenes/100_11/c3bo/05_10/2017-05-10-16-24-56_8.npy'
fname = '/local/data/scenes/100_11/c3bo/05_04/2017-05-04-19-07-38_10.npy'
fname = '/local/data/scenes/100_11/c3bo/05_09/2017-05-09-16-43-18_20.npy'
fname = '/local/data/scenes/100_11/mimo/05_15/2017-05-15-12-21-31_10.npy' #first stright then turn
fname = '/local/data/scenes/100_11/c3bo/05_04/2017-05-04-18-47-38_9.npy'
fname = '/home/ful7rng/projects/transition/data/selected_scenes_from_100_11.npy'
fname = '/home/ful7rng/projects/transition/data/244_scenes_for_test_from_80_new.npy'
sample_rate = 3
scenes = get_scenes(random_file=False,
                  min_time_interval=4,
                  max_time_interval=1e6,
                  sample_rate=sample_rate,
                  file_name=fname)[0]
idx = np.random.choice(np.arange(len(scenes)))
print(idx)
idx = 200
scene = scenes[idx]
map_ = scene.static_map
print(scene.hits.shape)

animate_scenes([scene])

load file: /home/ful7rng/projects/transition/data/244_scenes_for_test_from_80_new.npy
Found 244 scenes in total
48
12000000
Found 244.000000 scenes (= 100.00%) whose interval is in range (4.00, 1000000.00)
Found 0 scenes (= 0.000%) whose interval does not match number of hits.
61
(17, 32, 32)


In [187]:
animate_scenes([scenes[0]])

In [180]:
np.save(s_fname, scenes)

In [121]:
scene_copy = deepcopy(scene)
processed_scene_copy = BOFUMRealdata.scene_preprocessing(scene_copy)
animate_scenes([processed_scene_copy])

hit[22:29,26:32].sum()==1
observation at loc (25, 29) frame (3) is noise
hit[22:29,26:32].sum()==1
observation at loc (25, 29) frame (6) is noise
hit[22:29,26:32].sum()==1
observation at loc (25, 29) frame (7) is noise
hit[18:25,26:32].sum()==1
observation at loc (21, 29) frame (12) is noise
hit[15:22,11:18].sum()==1
observation at loc (18, 14) frame (13) is noise
hit[15:22,11:18].sum()==1
observation at loc (18, 14) frame (14) is noise


In [369]:

force_predict = True
simulated_data = False
naive_kwargs = {
                        'omega': 0.152265835237,
                        'extent': 5,
                        'noise_var': 0.676892668163,
                        'name': 'BOFUM',
                        'measurement_lost' : 8
                      }

kwargs = {
        'omega': 0.191234382275,
        'extent': 7,
        'noise_var': 0.644828151836,
        'name': 'BOFMP'}

ma_kwargs = {
                        'omega': 0.0263632138087,
                        'extent': 7,
                        'noise_var': 0.743921945138,
                        'name': 'BOFMP with moving average',
                        'measurement_lost': 8,
                        'keep_motion': True,
                        'window_size': 4,
                        'initial_motion_factor': 0.562946012048,
                        'keep_motion_factor': 0.706594184009
}

naive_bofum = naiveBOFUM(map_, simulated_data=simulated_data, **naive_kwargs)
bofum = conditionalBOFUM(map_, model_name,
                               simulated_data=simulated_data,
                               force_predict=force_predict,
                               acceleration_interpretation=True,
                               **kwargs)

# bofum_blur = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc blured',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                 nn_probs = bofum.nn_probs.copy(),
#                                acceleration_interpretation=True,
#                                blur_spatially=True,
#                                blur_var=0.83,
#                                blur_extent=5,
#                                **kwargs)
# bofum_blur.cnn_model = bofum.cnn_model

# bofum_blur_km = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc moving average',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                 nn_probs = bofum.nn_probs.copy(),
#                                acceleration_interpretation=True,
#                                blur_spatially=True,
#                                keep_motion=True,
#                                **kwargs)
# bofum_blur_km.cnn_model = bofum.cnn_model



# bofum_not_diagonal = conditionalBOFUM(map_, '03_DIAGONAL_FALSE_CONDITIONAL_TRUE',
#                                name = 'BOFUM acc. not diagonal',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                acceleration_interpretation=True,
#                                **kwargs)

# bofum_not_diagonal_blur = conditionalBOFUM(map_, '03_DIAGONAL_FALSE_CONDITIONAL_TRUE',
#                                name = 'BOFUM acc. not diagonal blurred',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                acceleration_interpretation=True,
#                                nn_probs = bofum_not_diagonal.nn_probs.copy(),
#                                blur_spatially=True,
#                                **kwargs)
# bofum_not_diagonal_blur.cnn_model = bofum_not_diagonal.cnn_model


# bofum_keep = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc keep naive',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                 nn_probs = bofum.nn_probs,
#                                acceleration_interpretation=True,
#                                keep_naive_bofum=True,
#                                naive_bofum_factor=0.5,
#                                **kwargs)
# bofum_keep.cnn_model = bofum.cnn_model

bofum_ma = conditionalBOFUM(map_, model_name,
                               simulated_data=simulated_data,
                               force_predict=force_predict,
                               nn_probs = bofum.nn_probs,
                               **ma_kwargs)
bofum_ma.cnn_model = bofum.cnn_model

# bofum_vm = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc. with velocity mask',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                nn_probs = bofum.nn_probs,
#                                velocity_mask=True,
#                                velocity_mask_factor=0.3,
#                                **kwargs)
# bofum_vm.cnn_model = bofum.cnn_model

# bofum_vm_blur = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc. with velocity mask, blur spatially',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                nn_probs = bofum.nn_probs,
#                                velocity_mask=True,
#                                velocity_mask_factor=0.3,
#                                blur_spatially=True,
#                                **kwargs)
# bofum_vm_blur.cnn_model = bofum.cnn_model


---------------------------------------------------------------------------
---------------------------------------------------------------------------

    Layer           Output shape         W shape             
1   Input           (None, 1, None, None)                     
2   Conv2D          (None, 8, None, None) (8, 1, 1, 1)        
3   Conv2D          (None, 12, None, None) (12, 8, 3, 3)       
4   Concat          (None, 20, None, None)                     
5   Conv2D          (None, 12, None, None) (12, 20, 3, 3)      
6   Concat          (None, 32, None, None)                     
7   Conv2D          (None, 12, None, None) (12, 32, 3, 3)      
8   Concat          (None, 44, None, None)                     
9   Conv2D          (None, 12, None, None) (12, 44, 3, 3)      
10  Concat          (None, 56, None, None)                     
11  Conv2D          (None, 12, None, None) (12, 56, 3, 3)      
12  Concat          (None, 68, None, None)                     
13  Conv2D         

In [370]:
models = []
models.append(naive_bofum)
models.append(bofum)
#models.append(bofum_blur)
# models.append(bofum_blur_km)
# models.append(bofum_not_diagonal)
# models.append(bofum_not_diagonal_blur)
models.append(bofum_ma)
# models.append(bofum_vm)
# models.append(bofum_keep)
# models.append(bofum_vm_blur)

In [345]:

ani = TrackingAnimRealdata(models, num_steps, scene,
                                plot_map=True,
                                plot_seen=True)

fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))
fig_size = (5 * 4, 5)
fig_3 = plt.figure(figsize=fig_size)
ma_ax = fig_3.add_subplot(141)
ma_plot = Plot(ma_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='moving average vel/ vel mask')
vel_ax = fig_3.add_subplot(142)
vel_plot = Plot(vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='predicted vel')
merge_vel_ax = fig_3.add_subplot(143)
merge_vel_plot = Plot(merge_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='merged velocity')
final_vel_ax = fig_3.add_subplot(144)
final_vel_plot = Plot(final_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='velocity')

ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
ani.fig.canvas.mpl_connect('key_press_event', lambda event: on_press(event, ani))

12

frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.562946012048
frame = 8
motion factor :0.397774378024
frame = 9
motion factor :0.28106506206
frame = 10
motion factor :0.198598938179
frame = 11
motion factor :0.140328854668
frame = 12
motion factor :0.0991555525571
frame = 13
motion factor :0.070062736749
frame = 14
motion factor :0.0495059223026
frame = 15
motion factor :0.034980596773
frame = 0
frame = 1


### draw keep motion velocities for thesis

In [371]:
from matplotlib.ticker import FuncFormatter, MaxNLocator

ani = TrackingAnimRealdata(models, num_steps, scene,
                                plot_map=True,
                                plot_seen=True)

fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))
fig_size = (8, 8)
fig_3 = plt.figure(figsize=fig_size)
ma_ax = fig_3.add_subplot(221)
ma_plot = Plot(ma_ax, bofum.ma_vel, 1, False, False, False, colorbar_on=None, title=r'$P(V_{ma})$')
vel_ax = fig_3.add_subplot(222)
vel_plot = Plot(vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on=None, title=r'$P(V_{pred})$')
merge_vel_ax = fig_3.add_subplot(223)
merge_vel_plot = Plot(merge_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on=None, title=r'$P(V)$')
merge_vel_ax.set_ylabel("BOFMP")
final_vel_ax = fig_3.add_subplot(224)
final_vel_plot = Plot(final_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on=None, title='$P(V)$')
final_vel_ax.set_ylabel("BOFMP with motion keeping")

    
xlabels = (np.arange(7) + np.array([-(7//2)])).tolist()
ylabels = xlabels

def format_fn_x(tick_val, tick_pos):
    if int(tick_val) in range(7):
        return xlabels[int(tick_val)]
    else:
        return ''

def format_fn_y(tick_val, tick_pos):
    if int(tick_val) in range(7):
        return ylabels[int(tick_val)]
    else:
        return ''
    
for ax in [ma_ax, vel_ax, merge_vel_ax, final_vel_ax]:
    ax.set_xticks(np.arange(.5, 7.5, 1.0))
    ax.set_yticks(np.arange(0.5, 7.5, 1.0))
    ax.xaxis.set_major_formatter(FuncFormatter(format_fn_x))
    #ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(FuncFormatter(format_fn_y))
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    if ax not in [merge_vel_ax, final_vel_ax]:
        ylabel = ax.set_ylabel(r'$V_y$', color='darkred', fontsize=12)
        ylabel.set_rotation(0)
        ax.yaxis.set_label_coords(-0.06, .95)
        ax.set_xlabel(r'$V_x$', color='darkred', fontsize=12)
        ax.xaxis.set_label_coords(1.05, -0.025)

c_x, c_y = 7 / 2., 7 / 2.

ma_ax.arrow(c_x, c_y , 1, -1, width=.05, head_width=.2, head_length=.3, fc='g', ec='g', zorder=15)
vel_ax.arrow(c_x, c_y , 1, 0, width=.05, head_width=.2, head_length=.3, fc='g', ec='g', zorder=15)
merge_vel_ax.arrow(c_x, c_y , 1, 0, width=.05, head_width=.2, head_length=.3, fc='g', ec='g', zorder=15)
final_vel_ax.arrow(c_x, c_y , 1, 0, width=.05, head_width=.2, head_length=.3, fc='g', ec='g', zorder=15)
final_vel_ax.arrow(c_x, c_y , 1, -1, width=.05, head_width=.2, head_length=.3, fc='g', ec='g', zorder=15)
# for ax in [vel_ax, merge_vel_ax, final_vel_ax]:
#     ax.set_xticks([])
#     ax.set_yticks([])
    
ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick_keep_motion(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
ani.fig.canvas.mpl_connect('key_press_event', lambda event: on_press(event, ani))

12

frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.562946012048
[12 23]
Click is in axes ax2
[[ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.]]
loc (12, 23) of model BOFUM has occupancy of 0.313751258906
loc (12, 23) of model BOFMP has occupancy of 0.0800594367367
loc (12, 23) of model BOFMP with moving average has occupancy of 0.925699409763


### tracking simulation data

In [276]:
scene_file = '/home/ful7rng/projects/transition/propagation/scenes/500_simulated_scenes_diagonal_for_testing_from_80_new.npy'
#scene_file = '/home/ful7rng/projects/transition/propagation/results/scenes_num_500_min_t_4_max_t_1000000.0_sample_rate_3_simulated_True_diagonal_True.npy'
simulated_scenes = True
scenes, return_flag = get_scenes(file_name=scene_file,
                                 simulated_scenes=simulated_scenes,
                                 max_scenes=None
                                 )

load file: /home/ful7rng/projects/transition/propagation/scenes/500_simulated_scenes_diagonal_for_testing_from_80_new.npy
Take 500 scenes for evaluation


In [277]:
import random
scene = random.sample(scenes, 1)[0]
num_steps = scene.hits.shape[0]

animate_scenes([scene])

In [287]:
animate_scenes([scenes[27]])

Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 389, in process
    proxy(*args, **kwargs)
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "<ipython-input-285-5e8a16001f9d>", line 19, in <lambda>
    ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
  File "<ipython-input-117-f4ea24fd9f56>", line 5, in onclick
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'


frame = 14
frame = 15
frame = 0
frame = 1
172
update model BOFUM
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
frame = 0
frame = 1
frame = 2
frame = 3
396
update model BOFUM
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
frame = 0
frame = 1
frame = 2
frame = 3
428
update model BOFUM
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
41
update model BOFUM
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 0
frame = 1
142
update model BOFUM
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame =

Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 389, in process
    proxy(*args, **kwargs)
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/matplotlib/cbook/__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "<ipython-input-285-5e8a16001f9d>", line 19, in <lambda>
    ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
  File "<ipython-input-117-f4ea24fd9f56>", line 5, in onclick
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'


frame = 6


In [245]:
scene.hits.shape

(16, 32, 32)

In [278]:
map_ = scene.static_map

In [244]:
def make_T_section(size):
    map_size = size
    map_ = np.ones((map_size, map_size), dtype=bool)
    lane_length = size // 2

    for j in range(map_size / 2):
        map_[j + 1:j + lane_length + 1, j] = 0
    for j in range(map_size / 2, map_size):
        map_[map_size - j:map_size - j + lane_length, j] = 0
    for j in range(map_size / 2 - lane_length / 2, map_size / 2 + lane_length / 2 + 1):
        map_[map_size / 2:map_size, j] = 0
    x, y = map_size - map_size / 4, map_size / 2
    return map_, x, y

In [59]:
map_, x, y = make_T_section(61)

In [60]:
scene_generator = naiveBOFUM(map_, name='BOFUM', simulated_data=True, noise_var=0.6)

In [61]:
scene_generator.initialize(np.random.randint(1, 4), num_steps, diagonal=True)
scene = scene_generator.trajs_to_scene()

sampling trajectories: 1/3


In [86]:
def on_press_simulation(event, animation):
    global scenes
    if event.key == 'n':
        #scene_generator.map = random.sample(scenes, 1)[0].static_map
        scene_generator.initialize(np.random.randint(1, 4), num_steps, diagonal=True)
        scene = scene_generator.trajs_to_scene()
#         scene = random.sample(scenes, 1)[0]
        animation.update(scene)

In [291]:

force_predict = True
simulated_data = False
naive_kwargs = dict(omega=0.0164534434234,
                  extent=7,
                  noise_var=0.555536471101,
                  measurement_lost=8,
                  verbose=False)

kwargs = dict(omega=0.0353639265597,
                  extent=7,
                  noise_var=0.372650880654,
                  measurement_lost=8,
                  verbose=False)

naive_bofum = naiveBOFUM(map_, name='BOFUM', simulated_data=simulated_data, **naive_kwargs)
bofum = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM with n. o. as acceleration',
                               simulated_data=simulated_data,
                               force_predict=force_predict,
                               acceleration_interpretation=True,
                               **kwargs)

bofum_blur = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM acc blured',
                               simulated_data=simulated_data,
                               force_predict=force_predict,
                                nn_probs = bofum.nn_probs.copy(),
                               acceleration_interpretation=True,
                               blur_spatially=True,
                               blur_var=1.09,
                               blur_extent=5,
                               **kwargs)
bofum_blur.cnn_model = bofum.cnn_model

# bofum_blur_km = conditionalBOFUM(map_, model_name,
#                                name = 'BOFUM acc moving average',
#                                simulated_data=simulated_data,
#                                force_predict=force_predict,
#                                 nn_probs = bofum.nn_probs.copy(),
#                                acceleration_interpretation=True,
#                                blur_spatially=True,
#                                keep_motion=True,
#                                **kwargs)
# bofum_blur_km.cnn_model = bofum.cnn_model





---------------------------------------------------------------------------
---------------------------------------------------------------------------

    Layer           Output shape         W shape             
1   Input           (None, 1, None, None)                     
2   Conv2D          (None, 8, None, None) (8, 1, 1, 1)        
3   Conv2D          (None, 12, None, None) (12, 8, 3, 3)       
4   Concat          (None, 20, None, None)                     
5   Conv2D          (None, 12, None, None) (12, 20, 3, 3)      
6   Concat          (None, 32, None, None)                     
7   Conv2D          (None, 12, None, None) (12, 32, 3, 3)      
8   Concat          (None, 44, None, None)                     
9   Conv2D          (None, 12, None, None) (12, 44, 3, 3)      
10  Concat          (None, 56, None, None)                     
11  Conv2D          (None, 12, None, None) (12, 56, 3, 3)      
12  Concat          (None, 68, None, None)                     
13  Conv2D         

In [292]:
models = []
models.append(naive_bofum)
models.append(bofum)
models.append(bofum_blur)
# models.append(bofum_blur_km)

In [293]:
ani = TrackingAnimRealdata(models, num_steps, scene,
                                plot_map=True,
                                plot_seen=True,
                                simulated_scenes=simulated_scenes)

fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))
fig_size = (5 * 4, 5)
fig_3 = plt.figure(figsize=fig_size)
ma_ax = fig_3.add_subplot(141)
ma_plot = Plot(ma_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='moving average vel/ vel mask')
vel_ax = fig_3.add_subplot(142)
vel_plot = Plot(vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='predicted vel')
merge_vel_ax = fig_3.add_subplot(143)
merge_vel_plot = Plot(merge_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='merged velocity')
final_vel_ax = fig_3.add_subplot(144)
final_vel_plot = Plot(final_vel_ax, bofum.ma_vel, 1, False, False, False, colorbar_on='occupancy_axes', title='velocity')

ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2, fig_3, ma_plot, vel_plot, merge_vel_plot, final_vel_plot))
ani.fig.canvas.mpl_connect('key_press_event', lambda event: on_press(event, ani))

14

frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12


In [ ]:
walls = set(map(lambda point: tuple(point), np.array(np.where(bofum.map)).T.tolist()))
print(walls)

In [403]:
trajectories = bofum.trajectories.copy()

In [406]:
trajectories[:, 2]

[autoreload of propagation.bofum failed: Traceback (most recent call last):
  File "/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/local/home/ful7rng/projects/transition/propagation/bofum.py", line 580
    class BOFUMRealdata(BOFUM):
                              ^
IndentationError: expected an indented block
]


array([[ 4, 28],
       [ 5, 27],
       [29, 16]])

In [416]:
trajectories = bofum.trajectories.copy()
num_steps = bofum.trajectories.shape[1]
gt_occupancy = np.zeros((num_steps, )+bofum.map.shape)
print(sqaure_trajs.shape)
distances = [(x, y) for x in [0, -1, 1] for y in [0, -1, 1]]
w, h = bofum.map.shape

for idx_1, traj in enumerate(trajectories):
    for idx_2, point in enumerate(traj):
        def get_neighbor(dist):
            x = min(max(0, point[0]+dist[0]), w-1)
            y = min(max(0, point[1]+dist[1]), h-1)
            return (x, y)
        neighbors = set(map(get_neighbor, distances))
        neighbors = neighbors.difference(neighbors.intersection(walls))
        for neighbor in neighbors:
            gt_occupancy[(idx_2, )+neighbor] = 1

        


(3, 41, 9)


In [365]:
from sklearn.metrics import f1_score
gt = np.random.randint(0, 2, 10)
print(gt)
pred = np.random.randint(0, 2, 10)
print(pred)
print(f1_score(gt, pred))

[0 1 0 1 1 1 1 0 1 1]
[0 1 1 1 0 1 1 0 0 1]
0.769230769231
ERROR! Session/line number was not unique in database. History logging moved to new session 269
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.5


12

frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor :0.108
frame = 11
motion factor :0.0648
frame = 12
motion factor :0.03888
frame = 13
motion factor :0.023328
frame = 14
motion factor :0.0139968
frame = 15
motion factor :0.00839808
frame = 16
motion factor :0.005038848
frame = 17
motion factor :0.0030233088
frame = 18
motion factor :0.00181398528
709
update model BOFUM
observation at loc (18, 27) frame (9) is noise
observation at loc (18, 27) frame (10) is noise
update model BOFUM with n. o. as acceleration
update model BOFUM acc blured
update model BOFUM acc moving average
update model BOFUM acc. not diagonal
update model BOFUM acc. not diagonal blurred
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.5
frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor :0.108
frame = 11
motion factor :0.0648
frame = 12
motion factor :0.03888
frame = 13
motion factor :0.023328
frame = 14


/local/home/ful7rng/.anaconda/envs/ful/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.5
frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor :0.108
frame = 11
motion factor :0.0648
frame = 12
motion factor :0.03888
frame = 13
motion factor :0.023328
frame = 14
motion factor :0.0139968
frame = 15
motion factor :0.00839808
frame = 16
motion factor :0.005038848
frame = 17
motion factor :0.0030233088
frame = 18
motion factor :0.00181398528
frame = 19
motion factor :0.001088391168
frame = 20
motion factor :0.0006530347008
frame = 21
motion factor :0.00039182082048
frame = 22
motion factor :0.000235092492288
frame = 23
motion factor :0.000141055495373
frame = 24
motion factor :8.46332972237e-05
frame = 25
motion factor :5.07799783342e-05
frame = 26
motion factor :3.04679870005e-05
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
motion factor :0.5
frame = 8
motion factor :0.3
frame = 9
motion factor :0.18
frame = 10
motion factor

In [278]:
kernels = models[1].kernels
w, h = kernels.shape[:2]

AttributeError: 'NoneType' object has no attribute 'shape'

ERROR! Session/line number was not unique in database. History logging moved to new session 267
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18


12

frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame = 0
frame = 1
frame = 2
frame = 3
frame = 4
frame = 5
frame = 6
frame = 7
frame = 8
frame = 9
frame = 10
frame = 11
frame = 12
frame = 13
frame = 14
frame = 15
frame = 16
frame = 17
frame = 18
frame

In [48]:
plot_4d_tensor(temp[2, 2])
plot_4d_tensor(temp[1, 1])

In [41]:
from utils import nd_gaussian

def blur_kernels_spatially(kernels, blur_extent=5, var=0.6):

    w, h = kernels.shape[:2]
    half_be = blur_extent //  2

    blurred = np.zeros_like(kernels)
    blurred = np.pad(blurred, ((half_be, half_be), (half_be, half_be), 
                               (0, 0), (0, 0), (0, 0), (0, 0)), 
                                mode='constant', constant_values=0)

    gaussian = nd_gaussian((blur_extent, blur_extent), (half_be, half_be), var)
    plt.figure()
    ax = plt.imshow(gaussian)
    plt.colorbar(ax)

    for i in range(w):
        for j in range(h):
            #print(i, j)
            temp = gaussian[..., None, None, None, None] * kernels[i, j]
            blurred[i:i+blur_extent, j:j+blur_extent] += temp
    blurred = blurred[half_be:half_be+w, half_be:half_be+h]
    # normalize to a probability distribution
    with np.errstate(divide='ignore', invalid='ignore'):
        blurred /= blurred.sum(axis=(4, 5), keepdims=True)
        blurred[~np.isfinite(blurred)] = 0
        
    return blurred

In [53]:
models = []


bofum_1 = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM w kernel blur',
                               simulated_data=simulated_data,
                               force_predict=False,
                               acceleration_interpretation=True,
                               **kwargs)


bofum_2 = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM w condi_prob blur',
                               simulated_data=simulated_data,
                               force_predict=False,
                               acceleration_interpretation=True,
                               **kwargs)

bofum_3 = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM w condi_prob blur',
                               simulated_data=simulated_data,
                               force_predict=False,
                               acceleration_interpretation=True,
                               **kwargs)



models.append(bofum)
models.append(bofum_1)
models.append(bofum_2)
models.append(bofum_3)
map(lambda model: model.initialize(scene), models)

blurred_kernels = blur_kernels_spatially(bofum.kernels)
bofum_1.kernels = blurred_kernels
blurred_condi = blur_kernels_spatially(bofum.conditional_probs)
bofum_2.conditional_probs = blurred_condi
bofum_2.construct_kernels()
blurred_condi = blur_kernels_spatially(bofum.conditional_probs, blur_extent=7, var=0.8)
bofum_3.conditional_probs = blurred_condi
bofum_3.construct_kernels()

In [43]:
(blurred_kernels == bofum_1.kernels).all()

True

In [44]:
(bofum.kernels == bofum_1.kernels).all()

False

In [45]:
(bofum_2.kernels == bofum_1.kernels).all()

False

In [54]:
# get cross entropy
results = np.zeros((len(models), num_steps), dtype=float)

In [55]:
for i in range(num_steps):
    for m, model in enumerate(models):
        model.tracking_step()
        results[m, i] = model.calc_cross_entropy()

In [56]:
results

array([[ 25.49073465,   3.84869801,   2.13929921,   3.93393667,
          1.78277712,   3.92495086,   5.44265805,   6.30542227,
          5.10214595,   4.77591112,   2.53913048,   1.76680371,
          2.73617617,   0.75845949,   2.92018203,   1.91264348,
          2.08302724,   0.79905669,   2.19960102,   1.50478909,
          1.23882981,   1.67931359,   3.96487629,   1.5758093 ,
          1.35595518,   0.76588525,   0.52078869,   1.07712621,
          1.26976321,   5.31276919,   4.41687738,   5.09051301,
          4.71810355,   3.50030789,   3.24112469,   0.68395647],
       [ 25.49073465,   3.81673963,   2.10372351,   3.82652506,
          1.73075608,   3.78575316,   5.36304209,   6.30201805,
          5.18764487,   4.72674817,   2.42158455,   1.94718851,
          2.92398964,   0.83711008,   3.29378999,   2.12195319,
          2.2661867 ,   0.86822227,   2.26942381,   1.66654572,
          1.2038297 ,   1.85901365,   3.2495976 ,   1.53127673,
          1.39344664,   0.90537844,   0

In [57]:
plt.figure()
for i in range(len(models)):
    plt.plot(results[i], label=models[i].name)
plt.legend()

### numerical issue of gaussian kernels

In [83]:
def get_gaussian_kernels(num_dim, increased_gaussian, fig):
    gaussian_kernels = np.zeros(([extent] * num_dim * 2))
    for idx in np.ndindex(*([extent] * num_dim)):
        raw_input()
        gaussian_kernels[tuple(idx)] = gaussian_kernel_at(idx, increased_gaussian, fig)
    return gaussian_kernels

In [102]:
def gaussian_kernel_at(idx, increased_kernel, fig):
    """ Return a gaussian kernel centered at idx. """
    print(idx)
    slices = map(lambda cor: slice(extent-cor-1, extent-cor+extent-1), idx)
    #slices = map(lambda cor: slice(cor, cor+extent), idx)
    print(slices)
    kernel = increased_kernel[slices].copy()
    # normalize
    kernel /= np.sum(kernel)
    
    plot_4d_tensor(kernel, fig=fig)
    plt.suptitle(idx)
    fig.canvas.draw()
    
    return kernel

In [60]:
def get_increased_kernel(extent, noise_var, num_dim):
    increased_extent = extent + extent // 2 * 2
    gaussian_kernel = nd_gaussian([increased_extent] * num_dim,
                                       center=[increased_extent // 2] * num_dim,
                                       cov=noise_var)
    return gaussian_kernel

In [103]:
extent = 5
num_dim = 4
noise_var = 0.6

In [62]:
increased_gaussian = get_increased_kernel(extent, noise_var, num_dim)

In [63]:
increased_gaussian.shape

(9, 9, 9, 9)

In [104]:
fig = plt.figure()
kernels = get_gaussian_kernels(num_dim, increased_gaussian, fig)


(0, 0, 0, 0)
[slice(4, 9, None), slice(4, 9, None), slice(4, 9, None), slice(4, 9, None)]

(0, 0, 0, 1)
[slice(4, 9, None), slice(4, 9, None), slice(4, 9, None), slice(3, 8, None)]

(0, 0, 0, 2)
[slice(4, 9, None), slice(4, 9, None), slice(4, 9, None), slice(2, 7, None)]

(0, 0, 0, 3)
[slice(4, 9, None), slice(4, 9, None), slice(4, 9, None), slice(1, 6, None)]


KeyboardInterrupt: 

In [23]:
fname = 'scenes_num_1000_min_t_3_max_t_4_sample_rate_3.npy'
folder = '/home/ful7rng/projects/transition/propagation/results/'
f = folder + fname
scenes = np.load(f)

In [59]:
data_folder = '/local/data/scenes/100_11'
scenes, return_flag = get_scenes(data_folder, 3, 4,
                                         max_scenes=1000,
                                         sample_rate=3,
                                         laser_fre=12)

Found 194 files in total
Found 8097 scenes in total
36
48
Found 1112 scenes (= 13.733%) whose interval is in range (3, 4)
Found 163 scenes (= 2.013%) whose interval does not match number of hits.
Take 1000 scenes for evaluation


In [31]:
x = np.arange(100)
x = x[::5]
x

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
       85, 90, 95])

In [52]:
x = np.random.randint(0, 10, (7,8))

In [50]:
?np.random.choice

In [55]:
import random
y = np.array(random.sample(x, 5))

In [56]:
y.shape

(5, 8)

In [57]:
scenes.shape

(1000,)

In [43]:
for i, scene in enumerate(scenes):
    if scene.hits.shape[0] == 1:
        print(i)
    #print(i)

In [42]:
max_scenes = 1000
if max_scenes is not None and len(scenes) > max_scenes:
    scenes = scenes[np.random.randint(0, len(scenes), max_scenes)]
    print("Take %d scenes for evaluation" % max_scenes)


Take 1000 scenes for evaluation


In [22]:
scene = scenes[7]
print(scene.hits.shape)
animate_scenes([scene])

(1, 32, 32)


In [12]:
scene.__dict__

{'end': 1494405454.7891603,
 'hits': array([[[ 0.,  0.,  1., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]]),
 'origin': array([ 11.4,  10.8]),
 'res': 0.19999999999999998,
 'seens': array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]]),
 'start': 1494405451.2425327,
 'static_map': array([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ..., False, False,  True],
        [ True,  T

In [6]:
scene_copy = deepcopy(scene)
animate_scenes([BOFUMRealdata.scene_preprocessing(scene_copy)])

In [60]:
for i, scene in enumerate(scenes):
    scene_copy = deepcopy(scene)
    try:
        BOFUMRealdata.scene_preprocessing(scene_copy)
    except IndexError:
        print(i)

In [68]:
x = np.random.randn(6)
idx = np.random.randint(0, 6, 5)

In [69]:
x

array([ 0.00715804,  1.55625278, -0.42154636,  2.62302897,  1.51321242,
        0.70077197])

In [70]:
type(idx)

numpy.ndarray

In [71]:
idx

array([4, 1, 5, 1, 2])

In [72]:
x = x[idx]

In [73]:
x

array([ 1.51321242,  1.55625278,  0.70077197,  1.55625278, -0.42154636])